# Database 2

In [1]:
import sqlite3
import pandas as pd

In [5]:
c = sqlite3.connect("movies.db")
c

In [6]:
pd.read_sql("select * from sqlite_master", c)

,type,name,tbl_name,rootpage,sql
0,table,movies,movies,2,"CREATE TABLE ""movies"" (\n""Title"" TEXT,\n ""Dir..."


In [7]:
pd.read_sql("select * from movies", c).head(15)

,Title,Director,Year,Runtime,Rating,Revenue
0,Guardians of the Galaxy,James Gunn,2014,121,8.1,333.13
1,Prometheus,Ridley Scott,2012,124,7.0,126.46
2,Split,M. Night Shyamalan,2016,117,7.3,138.12
3,Sing,Christophe Lourdelet,2016,108,7.2,270.32
4,Suicide Squad,David Ayer,2016,123,6.2,325.02
5,The Great Wall,Yimou Zhang,2016,103,6.1,45.13
6,La La Land,Damien Chazelle,2016,128,8.3,151.06
7,Mindhorn,Sean Foley,2016,89,6.4,0.00
8,The Lost City of Z,James Gray,2016,141,7.1,8.01
9,Passengers,Morten Tyldum,2016,116,7.0,100.01


In [8]:
def qry(sql, cap=10):
    return pd.read_sql(sql, c).head(cap)

In [9]:
qry("""
SELECT *
FROM movies
""")

,Title,Director,Year,Runtime,Rating,Revenue
0,Guardians of the Galaxy,James Gunn,2014,121,8.1,333.13
1,Prometheus,Ridley Scott,2012,124,7.0,126.46
2,Split,M. Night Shyamalan,2016,117,7.3,138.12
3,Sing,Christophe Lourdelet,2016,108,7.2,270.32
4,Suicide Squad,David Ayer,2016,123,6.2,325.02
5,The Great Wall,Yimou Zhang,2016,103,6.1,45.13
6,La La Land,Damien Chazelle,2016,128,8.3,151.06
7,Mindhorn,Sean Foley,2016,89,6.4,0.00
8,The Lost City of Z,James Gray,2016,141,7.1,8.01
9,Passengers,Morten Tyldum,2016,116,7.0,100.01


# Review: Simple Selections

### Which *movie* has the *highest rating*?

In [14]:
qry("""
SELECT Title, Rating
FROM movies
ORDER BY Rating DESC
LIMIT 1
""")


,Title,Rating
0,The Dark Knight,9.0


### Which *director* made the *shortest movie*?

In [19]:
qry("""
SELECT Director, Runtime
FROM movies
ORDER BY Runtime
LIMIT 1
""")

,Director,Runtime
0,Claude Barras,66


### Which *director* made the *highest-revenue movie*?

In [25]:
qry("""
SELECT Director, Revenue
FROM movies
ORDER BY Revenue DESC
LIMIT 1
""")

,Director,Revenue
0,J.J. Abrams,936.63


### Which *movie* had the *highest revenues* in *2016*?

In [31]:
qry("""
SELECT Title, Revenue
FROM movies
WHERE Year=2016
ORDER BY Revenue DESC
LIMIT 1
""")

,Title,Revenue
0,Rogue One,532.17


### Which *3 movies* had the *highest revenues* in *2016*?

In [33]:
qry("""
SELECT Title, Revenue
FROM movies
WHERE Year=2016
ORDER BY Revenue DESC
LIMIT 3
""")

,Title,Revenue
0,Rogue One,532.17
1,Finding Dory,486.29
2,Captain America: Civil War,408.08


### Which *3 movies* have the *highest rating-to-revenue ratios*?

Introduce `AS`

In [38]:
qry("""
SELECT Title, Revenue / Rating AS Ratio
FROM movies
ORDER BY Ratio DESC
LIMIT 3
""")

,Title,Ratio
0,Star Wars: Episode VII - The Force Awakens,115.633333
1,Avatar,97.501282
2,Jurassic World,93.168571


# Aggregate Queries

```
SUM, AVG, COUNT, MIN, MAX
```

### How many *movies* are there?

In [44]:
qry("""
SELECT COUNT(Title) AS Count
FROM movies
""")

,Count
0,998


### How many *directors* are there?

In [48]:
qry("""
SELECT COUNT(DISTINCT Director) AS Count
FROM movies
""")

,Count
0,643


### What is the *total revenue* of *all the movies*?

In [49]:
qry("""
SELECT SUM(Revenue) AS Total
FROM movies
""")

,Total
0,72215.45


### What is the *average rating* across *all movies*?

* v1: with `SUM` and `COUNT`
* v2: with `AVG`

In [50]:
qry("""
SELECT SUM(Rating) / COUNT(Rating) AS Avg
FROM movies
""")

,Avg
0,6.723447


In [51]:
qry("""
SELECT AVG(Rating) AS Avg
FROM Movies
""")

,Avg
0,6.723447


### What is the *average revenue* and *average runtime* of *all the movies*?

In [52]:
qry("""
SELECT AVG(Revenue) AS AvgRev, AVG(Runtime) AS AvgRun
FROM movies
""")

,AvgRev,AvgRun
0,72.36017,113.170341


### What is the *average runtime* for a *James Gunn* movie?

In [55]:
qry("""
SELECT Director, AVG(Runtime) AS AvgRun
FROM movies
WHERE Director="James Gunn"
""")

,Director,AvgRun
0,James Gunn,104.0


### What is the *average revenue* for a *Ridley Scott* movie?

In [56]:
qry("""
SELECT Director, AVG(Revenue) AS AvgRev
FROM movies
WHERE Director="Ridley Scott"
""")

,Director,AvgRev
0,Ridley Scott,89.8825


### *How many movies* were there in *2016*?

In [59]:
qry("""
SELECT COUNT(Year) AS "2016"
FROM movies
WHERE Year=2016
""")

,2016
0,296


### What *percentage* of the *total revenue* came from the *highest-revenue movie*?

In [63]:
qry("""
SELECT MAX(Revenue) / SUM(Revenue) * 100 AS HighestPercentRev
FROM movies
""")

,HighestPercentRev
0,1.296994


### What *percentage* of the *total revenue* came from the *highest-revenue movie* in *2016*?

In [64]:
qry("""
SELECT MAX(Revenue) / SUM(Revenue) * 100 AS HighestPercentRev
FROM movies
WHERE Year=2016
""")
# not total revenue? just 2016 revenue

,HighestPercentRev
0,4.746581


# GROUP BY Queries

```sql
SELECT ???, ??? FROM Movies
GROUP BY ???
```

### What is the *total revenue* per each *year*?

* v1: the amounts
* v2: the amounts, as labeled by year

In [67]:
qry("""
SELECT Year, SUM(Revenue) AS TotalRev
FROM movies
GROUP BY Year
""")

,Year,TotalRev
0,2006,3624.46
1,2007,4306.23
2,2008,5053.22
3,2009,5292.26
4,2010,5989.65
5,2011,5431.96
6,2012,6910.29
7,2013,7544.21
8,2014,7997.40
9,2015,8854.12


### *How many movies* were by each *director*?

In [71]:
qry("""
SELECT Director, COUNT(Title) AS MoviesMade
FROM movies
GROUP BY Director
ORDER BY MoviesMade DESC
""")

,Director,MoviesMade
0,Ridley Scott,8
1,Paul W.S. Anderson,6
2,Michael Bay,6
3,M. Night Shyamalan,6
4,David Yates,6
5,Zack Snyder,5
6,Woody Allen,5
7,Peter Berg,5
8,Martin Scorsese,5
9,Justin Lin,5


### What is the *average rating* for each *director*?

In [77]:
qry("""
SELECT Director, AVG(Rating) as AvgRating
FROM movies
GROUP BY Director
ORDER BY AvgRating DESC
""")

,Director,AvgRating
0,Nitesh Tiwari,8.80
1,Christopher Nolan,8.68
2,Olivier Nakache,8.60
3,Makoto Shinkai,8.60
4,Florian Henckel von Donnersmarck,8.50
5,Aamir Khan,8.50
6,Naoko Yamada,8.40
7,Damien Chazelle,8.40
8,Thomas Vinterberg,8.30
9,S.S. Rajamouli,8.30


### What is the *average runtime* for each *director*?

In [89]:
qry("""
SELECT Director, AVG(Runtime) as AvgRun
FROM movies
GROUP BY Director
ORDER BY AvgRun DESC
""")

,Director,AvgRun
0,Abdellatif Kechiche,180.0
1,Aamir Khan,165.0
2,Andrea Arnold,163.0
3,Maren Ade,162.0
4,James Cameron,162.0
5,Rajkumar Hirani,161.5
6,Nitesh Tiwari,161.0
7,Andrew Dominik,160.0
8,S.S. Rajamouli,159.0
9,Hong-jin Na,156.0


### How many *unique directors* created a movie in each *year*

In [96]:
qry("""
SELECT Year, COUNT(DISTINCT Director) AS UniqueDirectors
FROM movies
GROUP BY Year
""")

,Year,UniqueDirectors
0,2006,44
1,2007,51
2,2008,51
3,2009,51
4,2010,60
5,2011,63
6,2012,64
7,2013,88
8,2014,97
9,2015,127


# Combining GROUP BY with other CLAUSES

<img src="groupby.png">

### What is the *total revenue* of per *year*, in *recent* years?

In [100]:
qry("""
SELECT Year, SUM(Revenue) as TotalRev
FROM movies
GROUP BY Year
HAVING Year>2010
""")

,Year,TotalRev
0,2011,5431.96
1,2012,6910.29
2,2013,7544.21
3,2014,7997.40
4,2015,8854.12
5,2016,11211.65


### Which *directors* have had the *largest number of movies* earning *over 100M dollars*?

In [106]:
qry("""
SELECT Director, COUNT(Title) AS NumOfMovies
FROM movies
WHERE Revenue>100
GROUP BY Director
ORDER BY NumOfMovies DESC
""")

,Director,NumOfMovies
0,David Yates,6
1,J.J. Abrams,5
2,Zack Snyder,4
3,Ridley Scott,4
4,Paul Feig,4
5,Michael Bay,4
6,Justin Lin,4
7,Francis Lawrence,4
8,Dennis Dugan,4
9,Christopher Nolan,4


### Which *three* of the *directors* have the *greatest average rating*?

In [110]:
qry("""
SELECT Director, AVG(Rating) AS AvgRating
FROM movies
GROUP BY Director
ORDER BY AvgRating DESC
LIMIT 3
""")

,Director,AvgRating
0,Nitesh Tiwari,8.80
1,Christopher Nolan,8.68
2,Olivier Nakache,8.60


### Which *three* of the *directors* have the *greatest average rating*?

Why is the above question maybe not the best to ask?

### Which *three* of the *directors* have the *greatest average rating* over at *least three movies*?

In [119]:
qry("""
SELECT Director, AVG(Rating) AS AvgRating, COUNT(Title)
FROM movies
GROUP BY Director
HAVING COUNT(Title) > 3
ORDER BY AvgRating DESC
LIMIT 3
""")

,Director,AvgRating,COUNT(Title)
0,Christopher Nolan,8.68,5
1,Martin Scorsese,7.92,5
2,Quentin Tarantino,7.90,4


Need filtering BEFORE and AFTER the GROUP operations

<img src="pipeline.png">

# WHERE vs. HAVING

* WHERE: filter rows in original table
* HAVING: filter groups

### Repeat: Which *three* of the *directors* have the *greatest average rating* over at *least three movies*?

<img src="having.png">

### Which *directors* have had *more than 5 movies* that have been *since 2014*?

In [123]:
qry("""
SELECT Director, COUNT(Title) AS NumMovies
FROM movies
WHERE Year>2010
GROUP BY Director
HAVING COUNT(Title) > 3
""")

,Director,NumMovies
0,Antoine Fuqua,4
1,Denis Villeneuve,4
2,Michael Bay,4
3,Mike Flanagan,4
4,Paul Feig,4
5,Paul W.S. Anderson,4
6,Peter Berg,4
7,Ridley Scott,4
8,Woody Allen,4


### Which *directors* have had more than *five* movies with runtimes under *100* minutes

In [127]:
qry("""
SELECT Director, COUNT(Title) as NumMovies
FROM movies
WHERE Runtime<100
GROUP BY Director
HAVING COUNT(Title) > 3
""")

,Director,NumMovies
0,Woody Allen,4


In [8]:
c.close()